In [1]:
import numpy as np
import cv2
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import PIL
from PIL import Image
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

def data_loader(data_dir,
                batch_size,
                random_seed,
                valid_size,
                test_size,
                shuffle):
  
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            normalize,
    ])

    # load the dataset
    dataset = datasets.ImageFolder(
        root=data_dir, transform=transform
    )

    # split into training, validation, and testing sets
    num_samples = len(dataset)
    indices = list(range(num_samples))
    split_valid = int(np.floor(valid_size * num_samples))
    split_test = int(np.floor(test_size * num_samples))
    split_train = num_samples - split_valid - split_test

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx = indices[:split_train]
    valid_idx = indices[split_train:split_train + split_valid]
    test_idx = indices[split_train + split_valid:]

    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    valid_dataset = torch.utils.data.Subset(dataset, valid_idx)
    test_dataset = torch.utils.data.Subset(dataset, test_idx)

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=shuffle
    )
    valid_loader = DataLoader(
        valid_dataset, batch_size=batch_size, shuffle=shuffle
    )
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=shuffle
    )

    return (train_loader, valid_loader, test_loader, train_dataset, valid_dataset, test_dataset)


In [3]:
train_loader, valid_loader, test_loader, train_dataset, valid_dataset, test_dataset= data_loader(data_dir='/home/stud1/Desktop/VGG16_Train/Dataset Generated',
                                                      batch_size=64,
                                                      random_seed=42,
                                                      valid_size=0.1,
                                                      test_size=0.1,
                                                      shuffle=True)


In [4]:
class VGG16(nn.Module):
    def __init__(self, num_classes=5):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(

            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
       
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
        
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
        
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
        
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
        
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.maxpool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(4608, 4096),
            nn.ReLU(),
        
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
        
            nn.Linear(4096, num_classes))
        self.gradient = None
        self.flat=nn.Flatten()
    def forward(self, x):
        x = self.features(x)
        if x.requires_grad :
            h=x.register_hook(self.activations_hook)
        x = self.maxpool(x)
        x=self.flat(x)
        x = self.classifier(x)
        return x
    def activations_hook(self,grad):
        self.gradient=grad 
    def get_activation_gradients(self):
       return self.gradient
    def get_activation(self,x):
       return self.features(x)
    

In [ ]:
num_classes = 5
num_epochs = 2
batch_size = 16
learning_rate = 0.005

model = VGG16(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(train_loader)



for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the validation images: {} %'.format( 100 * correct / total)) 

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the test images: {} %'.format(100 * correct/total))